## Community Detection-based Node Importance for Identifying Coding Genes

This notebook contains code for identifying important nodes in the cancer network. The cancer network was built in [1]. In this work, a community detection algorithm is used to partition the cancer network into communities and a centrality-based metric is implemented to compute the importance of each node in the network [2]. Specifically, the well-known Louvain algorithm is used for detecting communities, and a centrality-based metric is used to compute the importance of each node with respect to its community. 

The following steps are performed to compute the node importance and validate the important nodes,
1. Build an undirected graph G using the edge list of the cancer network.
2. Partition the graph into communities using the Louvain algorithm.
3. Compute eigenvectors of the adjancency matrix of graph G.
4. Compute node importance for each node in G using a centrality-based metric.
5. Sort the coding genes in the cancer network in descending order.
6. Validate the coding genes with gold standard CGC.

### References:
1. [Pham, Vu VH, Lin Liu, Cameron P. Bracken, Gregory J. Goodall, Qi Long, Jiuyong Li, and Thuc D. Le. "CBNA: A control theory based method for identifying coding and non-coding cancer drivers." PLoS Computational Biology 15, no. 12 (2019).](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007538#sec009)
2. [Wang, Y., Di, Z., & Fan, Y. (2011). Identifying and characterizing nodes important to community structure using the spectrum of the graph. PloS one, 6(11).](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0027418)

In [1]:
%load_ext autoreload
%autoreload 2

# Standard libraries
from os.path import join

# Libraries for graphs
import community
import networkx as nx

# Libraries for matrix computations
import numpy as np
import pandas as pd

# Libraries for sparse matrices and eigenvectors
from scipy.sparse import diags
from scipy.sparse.linalg import eigs

In [2]:
def compute_node_importance(n_nodes, n_communities, eig_vectors):
    """
    Computes node importance using a centrality-based metric which uses the eigenvectors of the adjacency matrix.
    Args:
        n_nodes: Number of nodes in the graph.
        n_communities: Number of communities estimated.
        eig_vectors: np.ndarray of size n x c where n is number of nodes and c is the number of communities.
        The eigenvectors are computed from the adjacency matrix and are stored as columns.
    Returns: 
        np.array: A numpy array containing the node importance score normalized by the
        number of communities.
    """
    p_k = [0] * n_nodes
    I = [0] * n_nodes
    for k in range(n_nodes):
        for i in range(n_communities):
            p_k[k] += eig_vectors[k, i]**2/np.sum(eig_vectors[:, i]**2)
        I[k] = p_k[k]/n_communities
    return np.array(I)

In [3]:
# Read cancer network data
DATA_DIR = "/home/mandar/Data/NCSU/CBNA/cbna-community-detection/Data/"
OUT_DIR = "/home/mandar/Data/NCSU/CBNA/cbna-community-detection/Output/"

In [4]:
cancer_network = pd.read_csv(DATA_DIR + "pVal_cancer_network.csv")

In [5]:
cancer_network.shape

(128264, 2)

### Step 1: Build an undirected graph G using the edge list of the cancer network

In [6]:
# Build graph by reading the edge list of the cancer network
G = nx.from_pandas_edgelist(cancer_network, source="cause", target="effect")

In [7]:
print("Number of nodes: {0}, and number of edges: {1}".format(len(G.nodes), len(G.edges)))

Number of nodes: 7357, and number of edges: 125867


### Step 2: Partition the graph into communities using the Louvain algorithm

In [8]:
# Run community detection algorithm on graph G
partition = community.best_partition(G)

In [38]:
k_clique_communities = nx.algorithms.community.k_clique_communities(G, k=2)

In [ ]:
list(k_clique_communities)[0]

In [31]:
n_communities = len(set(partition.values()))
print(f"Number of communities in G: {n_communities}")

Number of communities in G: 12


### Step 3: Compute eigenvectors of the adjancency matrix of graph G

In [10]:
# Create adjacency matrix of graph G
A = nx.adjacency_matrix(G)

In [11]:
# Compute eigenvalues and eigenvectors of the adjacency matrix A
adj_eig_vals, adj_eig_vecs = eigs(A.toarray().astype(float), k=n_communities, which='LR')

### Step 4: Compute node importance for each node in G using a centrality-based metric

In [12]:
# Compute node importance index I using the eigenvectors of the adjacency matrix
I = compute_node_importance(G.number_of_nodes(), n_communities, np.real(adj_eig_vecs))

## Select top-K important nodes in the network

### Step 5: Sort the coding genes in the cancer network in descending order

In [13]:
# Get list of all nodes in the cancer network
nodes = list(G.nodes)

In [14]:
# Sort the nodes by their importance score in descending order
important_nodes = [nodes[i] for i in np.argsort(I)[::-1]]
node_importance = np.sort(I)[::-1]

In [15]:
# Build a dataframe with two columns, one containing node name and second containing importance score
node_importance_df = pd.concat([pd.Series(important_nodes), pd.Series(node_importance)], axis=1)

In [16]:
node_importance_df.columns = ['node', 'importance']

In [17]:
# Print shape of the node importance dataframe
node_importance_df.shape

(7357, 2)

In [18]:
node_importance_df.head()

,node,importance
0,hsa-miR-30e-5p,0.018119
1,hsa-miR-23b-5p,0.015602
2,hsa-miR-23a-5p,0.015300
3,hsa-miR-16-5p,0.013795
4,hsa-miR-92a-1-5p,0.013278


In [19]:
node_importance_df.to_csv(OUT_DIR + 'community_detection_node_importance.csv')

### Step 6: Validate the coding genes with gold standard CGC

In [20]:
# Load gold standard CGC
gold_standard_cgc_df = pd.read_csv(join(DATA_DIR, "Census_allFri Sep 28 07_39_37 2018.tsv"), sep="\t")

In [21]:
gold_standard_cgc = gold_standard_cgc_df["Gene Symbol"]

In [22]:
# Load mRNAs column names from BRCA_matchedData RData file
mRNAs_data = pd.read_csv(join(DATA_DIR, "mRNAs_data.csv"))

In [23]:
# Find coding genes in the cancer network
# Perform intersection of the nodes in the cancer network and the mRNAs
coding_genes = node_importance_df.loc[node_importance_df.node.isin(mRNAs_data.columns), ]

In [24]:
coding_genes.shape

(5922, 2)

In [25]:
# Select top-K coding genes and save the results to csv files
top_k = [50, 100, 150, 200]
for K in top_k:
    top_100_coding_genes = coding_genes.iloc[:K, ].copy()
    coding_genes_gold_standard = top_100_coding_genes.loc[top_100_coding_genes.node.isin(gold_standard_cgc)]
    print("Coding genes in top-{0}: {1}".format(K, coding_genes_gold_standard.shape[0]))
    coding_genes_gold_standard.to_csv(join(OUT_DIR, "top_{0}_coding_genes_gold_standard_validation.csv".format(K)))

Coding genes in top-50: 25
Coding genes in top-100: 39
Coding genes in top-150: 53
Coding genes in top-200: 65
